In [ ]:
# 지오코더 설치 : https://geocoder.gimi9.com/

%pip install gimi9-geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from gimi9_geocoder.client import GeocoderClient
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 초기화
MY_TOKEN = "8f9a4803279634a86e0638339eb3b8b80cb33f90"

geocoder = GeocoderClient(token=MY_TOKEN)

In [ ]:
df = pd.read_csv('data/addr.csv', encoding='utf-8')
df.head(5)

,시군구,구,동,본번,부번,좌표X,좌표Y
0,서울특별시 강남구 개포동,강남구,개포동,658,1,127.05721,37.476763
1,서울특별시 강남구 개포동,강남구,개포동,652,0,127.0559898,37.4838941
2,서울특별시 강남구 개포동,강남구,개포동,12,2,127.0766243,37.4962963
3,서울특별시 강남구 개포동,강남구,개포동,141,0,127.0585214,37.4800022
4,서울특별시 강남구 개포동,강남구,개포동,187,0,127.0680279,37.4878016


In [6]:
df_xy = df.copy()

In [7]:
# 1. 좌표X, 좌표Y가 Unknown이고 본번이 0이 아닌 행만 필터링
mask = ((df_xy['좌표X'] == 'Unknown') | (df_xy['좌표Y'] == 'Unknown')) & (df_xy['본번'] != 0)
df_unknown = df_xy[mask].copy()

print(f"전체 데이터: {len(df_xy):,}개")
print(f"처리 대상: {len(df_unknown):,}개")

# 2. addresses 리스트 만들기
addresses = []
indices = []  # 나중에 좌표를 업데이트할 인덱스 저장

for idx, row in df_unknown.iterrows():
    # 부번이 0인 경우: "시군구 + 본번"
    if row['부번'] == 0:
        address = f"{row['시군구']} {row['본번']}"
    # 부번이 0이 아닌 경우: "시군구 + 본번 + '-' + 부번"
    else:
        address = f"{row['시군구']} {row['본번']}-{row['부번']}"
    
    addresses.append(address)
    indices.append(idx)

print(f"주소 생성 완료: {len(addresses):,}개")

for i in range(min(5, len(addresses))):
    print(f"   {i+1}. {addresses[i]}")

# 3. geocoder로 좌표 조회 (배치 처리 - 2,000개씩)
print(f"\n좌표 조회 중 (배치 처리)...")
BATCH_SIZE = 2000
total_success = 0
total_fail = 0

for batch_start in range(0, len(addresses), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, len(addresses))
    batch_addresses = addresses[batch_start:batch_end]
    batch_indices = indices[batch_start:batch_end]
    
    print(f"\n배치 {batch_start//BATCH_SIZE + 1}: {batch_start+1}~{batch_end} ({len(batch_addresses):,}개)")
    
    # geocoder 호출
    gdf = geocoder.geocode(batch_addresses)
    print(f"응답: {len(gdf):,}개")
    
    # 좌표 업데이트
    if 'address' in gdf.columns:
        # address 정보로 매칭
        batch_success = 0
        for i, address in enumerate(batch_addresses):
            matching_rows = gdf[gdf['address'] == address]
            if len(matching_rows) > 0:
                idx = batch_indices[i]
                df_xy.loc[idx, '좌표X'] = matching_rows.iloc[0]['geometry'].x
                df_xy.loc[idx, '좌표Y'] = matching_rows.iloc[0]['geometry'].y
                batch_success += 1
        
        total_success += batch_success
        total_fail += len(batch_addresses) - batch_success
        print(f"실패: {len(batch_addresses) - batch_success:,}개")
    else:
        # address 정보가 없으면 순서대로 매칭
        batch_success = 0
        for i in range(min(len(gdf), len(batch_indices))):
            idx = batch_indices[i]
            if gdf.iloc[i]['geometry'] is not None:
                df_xy.loc[idx, '좌표X'] = gdf.iloc[i]['geometry'].x
                df_xy.loc[idx, '좌표Y'] = gdf.iloc[i]['geometry'].y
                batch_success += 1
        
        total_success += batch_success
        total_fail += len(batch_indices) - batch_success
        print(f"실패: {len(batch_indices) - batch_success:,}개")

# 4. 최종 결과
print(f"좌표 있음: {((df_xy['좌표X'] != 'Unknown') & (df_xy['좌표Y'] != 'Unknown')).sum():,}개")
print(f"좌표 없음: {((df_xy['좌표X'] == 'Unknown') | (df_xy['좌표Y'] == 'Unknown')).sum():,}개")

전체 데이터: 8,961개
처리 대상: 8,203개
주소 생성 완료: 8,203개
   1. 서울특별시 강남구 개포동 189
   2. 서울특별시 강남구 개포동 185
   3. 서울특별시 강남구 개포동 655-1
   4. 서울특별시 강남구 개포동 649
   5. 서울특별시 강남구 개포동 1164-30

좌표 조회 중 (배치 처리)...

배치 1: 1~2000 (2,000개)
응답: 2,000개
실패: 0개

배치 2: 2001~4000 (2,000개)
응답: 2,000개
실패: 0개

배치 3: 4001~6000 (2,000개)
응답: 2,000개
실패: 0개

배치 4: 6001~8000 (2,000개)
응답: 2,000개
실패: 0개

배치 5: 8001~8203 (203개)
응답: 203개
실패: 0개
좌표 있음: 8,960개
좌표 없음: 1개


In [ ]:
df_xy.to_csv('data/addr_xy.csv', index=False, encoding='utf-8')